In [36]:
import openpyxl
import pyomo.environ as pe
import numpy as np
import pandas as pd

t=20
power_limit = 1200

In [37]:
#################################################################################################################
# Objetivo: Ler do excel "Info" informaçoes sobre cada load:
#
# ->Index: Indice de cada carga (1 a 15);
# ->Classificação em termos de flexibilidade: 0 (não flexivel), 1 (flexivel em potencia) e 2 (flexivel no tempo);
# ->T_ON: Tempo máximo que a carga deve estar ligada;
# ->T_OFF: Tempo máximo que a carga deve estar desligada;
# ->T_start: Modo no primeiro periodo: 0 (desligada) ou 1 (ligada);
# ->Power: Potência de cada carga quando está ligada.
#
#################################################################################################################

# load the workbook
workbook = openpyxl.load_workbook('Info.xlsx')

# select the worksheet
worksheet = workbook['Lds_inputs']

# read Index of cells
column = worksheet['B']
Index = []
for cell in column:
    if cell.row == 1:
        continue
    Index.append(cell.value)

# read Classification of cells
column = worksheet['C']
Class = []
for cell in column:
    if cell.row == 1:
        continue
    Class.append(cell.value)

# read T_ON of cells
column = worksheet['D']
T_ON = []
for cell in column:
    if cell.row == 1:
        continue
    T_ON.append(cell.value)

# read T_OFF of cells
column = worksheet['E']
T_OFF = []
for cell in column:
    if cell.row == 1:
        continue
    T_OFF.append(cell.value)

# read T_start of cells
column = worksheet['F']
T_start = []
for cell in column:
    if cell.row == 1:
        continue
    T_start.append(cell.value)

# read Power
column = worksheet['G']
Power = []
for cell in column:
    if cell.row == 1:
        continue
    Power.append(cell.value)

# print the values
print(f"Index - {(Index)}")
print(f"Classificação em termos de flexibilidade - {(Class)}")
print(f"T_ON - {(T_ON)}")
print(f"T_OFF - {(T_OFF)}")
print(f"Modo de começo - {(T_start)}")
print(f"Potência - {(Power)}")



Index - [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Classificação em termos de flexibilidade - [2, 2, 2, 1, 1, 1, 2, 2, 2, 0, 2, 2, 0, 0, 2]
T_ON - [3, 8, 4, 40, 40, 40, 15, 6, 38, 50, 7, 10, 3, 10, 32]
T_OFF - [4, 4, 10, 0, 0, 0, 4, 4, 65, 0, 45, 40, 0, 0, 60]
Modo de começo - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Potência - [90, 90, 90, 10, 10, 10, 590, 500, 180, 192, 300, 300, 1000, 2, 2000]


In [38]:
#Declaração de variáveis:
#Function_weight
cont_on = [0] * len(Index)
cont_off = [0] * len(Index)
counter_on = [0] * len(Index)
counter_off = [0] * len(Index)
compare = [0] * len(Index)
vector_x = [0] * len(Index)
ON_OFF_vector = [0] * len(Index)
weight = [10] * len(Index)
estado = [0] * len(Index) #(0=desligado, 1=ligado)
update =0

#Function_soma e Function_soma_opti
soma=[0] * t
soma_optim=[0] * t

Peso_max=0

rows = t
cols = len(Index)
weight_matrix = [['' for j in range(cols)] for i in range(rows)]
weight_before = [['' for j in range(cols)] for i in range(rows)]
ON_OFF_before = [['' for j in range(cols)] for i in range(rows)]
ON_OFF_matrix = [['' for j in range(cols)] for i in range(rows)]

In [39]:

###########################################################################################################
# Objetivo: Cria matrix ON/OFF de acordo com as especificaçoes do excel Info (lido anteriormente),
#           irá representar um dataset de entrada.
#
#           -> Cargas flexiveis no tempo (index=1,2,3,7,8,9,11,12,15): ligam e desligam alternadamente de acordo 
#              com T_ON e T_OFF;
#           -> Cargas flexiveis em potencia (index=4,5,6): uma vez que não faz sentido terem um T_OFF, 
#              eu assumo que ligam às 07:00 , 19:00 , 20:00 e 21:00, ficando ligadas de acordo com T_ON;
#           -> Cargas não flexiveis (index=10,13,14): São consideradas Fixed Loads, assumo que são ligadas 
#              às 13:00 e 20:00, permanendo ligadas até T_ON.
#
# Variaveis: ON_or_OFF (0=OFF, 1=ON).
###########################################################################################################

def function_dataset(i,update, ON_OFF_before):

    if (update==1):
        for j in range(len(Index)):
            if (ON_OFF_vector[j] == 1 and ON_OFF_before[i][j] == 0):
                estado[j]=1
                cont_off[j] = 0 
                cont_on[j] = 1
                

            elif (ON_OFF_vector[j] == 0 and ON_OFF_before[i][j] == 1):
                estado[j]=0
                cont_off[j] = 1 
                cont_on[j] = 0
        



    elif (update==0):
        if (i==0):
            for j in range(len(Index)): 
                if (T_start[j] == 1):           #Se começam ON:
                    ON_OFF_vector[j] = 1
                    estado[j]= 1
            
                elif (T_start[j] == 0):         #Se começam OFF:
                    ON_OFF_vector[j] = 0
                    estado[j]= 0
                


        for j in range(len(Index)):

            #Cargas flexiveis em tempo:
            if (Class[j]==2):
                if estado[j] == 1:                              #Se começam ON ou se estão ON:
                    if cont_on[j] < T_ON[j]:                    #Enquanto o contador não atinge T_ON, a carga continua ON
                        ON_OFF_vector[j] = 1
                        cont_on[j] += 1
                        estado[j]=1
                    else:                                       #Quando o contador atinge T_ON, então a carga fica OFF
                        cont_on[j] = 0 
                        ON_OFF_vector[j] = 0
                        cont_off[j] = 1
                        estado[j]=0

                elif estado[j] == 0:                            #Se começam OFF ou se estão OFF:
                    if cont_off [j] < T_OFF[j]:                 #Enquanto o contador não atinge T_OFF, a carga continua OFF
                        ON_OFF_vector[j] = 0
                        cont_off[j] += 1
                        estado[j]=0
                    else:                                       #Quando o contador atinge T_OFF, então a carga fica ON
                        cont_off[j] = 0 
                        ON_OFF_vector[j] = 1
                        cont_on[j] = 1
                        estado[j]=1
    
            #Cargas flexiveis em potência ou não flexiveis:
            elif (Class[j]== 0 or Class[j]== 1):
                if estado[j] == 1:                              #Se começam ON ou se estão ON
                    if cont_on[j] < T_ON[j]:                    #Enquanto o contador não atinge T_ON, a carga continua ON
                        ON_OFF_vector[j] = 1
                        cont_on[j] += 1
                        estado[j]=1
                    else:                                       #Quando o contador atinge T_ON, então a carga fica OFF
                        cont_on[j] = 0
                        ON_OFF_vector[j] = 0
                        cont_off[j] = 1
                        estado[j]=0

                elif estado[j] == 0 :                           #Se começam OFF ou se estão OFF
                    if ((Class[j]== 1) and (i==7 or i==19*60 or i==20*60 or i==21*60)) or ((Class[j]== 0) and (i==13*60 or i==20*60)): 
                    #Ligar cargas com flex. em potencia (luzes) às 07:00 , 19:00 , 20:00 e 21:00; 
                    #Ligar cargas não flexiveis as 13:00 e 20:00;
                        ON_OFF_vector[j] = 1
                        cont_on[j] += 1
                        estado[j] = 1
                    else:
                        ON_OFF_vector[j] = 0
                        estado[j] = 0
                        cont_on[j] = 0
                        cont_off[j] = 1

        return (ON_OFF_vector)
  



In [40]:
###########################################################################################################
# Objetivo: Está dividido em duas partes:
#           1) Faz update dos pesos apos a optmização;
#           2) Calcula os pesos para cada carga antes da otimização;
#          
#           Os pesos são de 0 a 10 e medem o comforto.
#           Caso a carga esteja OFF, os pesos vão de 1 a 10 (prioridade aumenta, deve-se ligar a carga);
#           Caso a carga esteja ON, os pesos vão de 10 a 1 (prioridade diminui, pode-se desligar a carga);
#           Em relação aos updates, o vetor "compare" serve para indicar quais das cargas foram alteradas;
#
# Nota: Em tempo real, apenas queremos cortar as cargas que estão ON com prioridades próximas de 1;
#       Deste modo, os pesos calculados quando as cargas estão OFF não são relevantes;
#       No entanto, os pesos foram calculados na mesma para cargas OFF, pois vai dar jeito para Hour-Ahead;
#       
# Variáveis: Peso [][]     
###########################################################################################################

def function_weight (i,ON_OFF_vector, update, weight_before, ON_OFF_matrix):
    peso =[10] * len(Index)
    
    #Update dos pesos apos a optimização
    if update == 1:
        for j in range(len(Index)): 

            peso[j] = weight_before[i][j]

            if Class[j] == 2 and ON_OFF_vector[j]==1 and ON_OFF_matrix[i-1][j]==0 :                                      #Se tiver sido ligada, então peso = 10
                peso [j] = 10
                counter_off[j] =0
                counter_on[j] =1

            elif Class[j] == 2 and ON_OFF_vector[j]==1 and ON_OFF_matrix[i-1][j]==1: 
                if weight_matrix[i-1][j] == 1:                                                            #Se tiver sido desligada, então peso = 1
                    peso[j] = 1
                    counter_on[j] = T_ON[j] +1
                

            elif Class[j] == 2 and ON_OFF_vector[j]==0 and ON_OFF_matrix[i-1][j]==1 :                                      #Se tiver sido ligada, então peso = 10
                peso [j] = 1
                counter_off[j] =1
                counter_on[j] =0

            elif Class[j] == 2 and ON_OFF_vector[j]==0 and ON_OFF_matrix[i-1][j]==0: 
                if weight_matrix[i-1][j] == 10 :                                                          #Se tiver sido desligada, então peso = 1
                    peso[j] = 10
                    counter_off[j] = T_OFF[j] +1
            
            elif Class[j] == 0 or Class[j] == 1:
                peso[j] = -1
            
                                                            #Se não tiver sido alterado, então o peso mantem-se
       
        return(peso)
             

    #Calculo dos pesos antes da optimização
    elif update == 0:
        for j in range(len(Index)):                                                    
            if Class[j] ==2 and ON_OFF_vector[j] == 1 :                              #Carga flexivel em tempo e ON:
                if counter_on[j] <= T_ON[j] - 1 :                                      #Enquanto contador < T_ON 
                    peso[j] = 10 - (counter_on[j] * 9/(T_ON[j] - 1))                   #Peso decresce de 10 a 1
                    counter_on[j] += 1
                    if counter_on[j] == T_ON[j]:                                       #Quando contador = T_ON 
                        peso[j] = 1                                                 #Peso = 1 = prioridade minima, posso desligar
                        counter_on[j] = 0
                elif counter_on[j] > T_ON[j]: 
                    peso[j] = 1
                     
                 
            elif Class[j] ==2 and ON_OFF_vector[j] == 0 :                            #Carga flexivel em tempo e OFF:
                if counter_off[j] <= T_OFF[j]-1 :                                      #Enquanto contador < T_OFF 
                    peso[j] = 1 + ((9/(T_OFF[j]-1)) * counter_off[j] )                 #Peso cresce de 1 a 10
                    counter_off[j] += 1
                    if counter_off[j] == T_OFF[j]:                                     #Quando contador = T_OFF 
                        peso[j] = 10                                                #Peso = 10 = prioridade maxima, devo ligar
                        counter_off[j] =0   
                elif counter_off[j] > T_OFF[j]: 
                    peso[j] = 10 

            elif Class[j] == 0 or Class[j] == 1:
                peso[j] = -1


        for j in range(len(Index)):
            peso[j]=round(peso[j], 3)
            
        return (peso)



In [41]:
###############################################################################
# Objetivo: Para cada periodo, soma a potência das cargas ON.
###############################################################################

def function_soma (i, ON_OFF_vector):
    soma=0
    aux=0
    sum_power=[0] * t
    
    for j in range(len(Index)):     
        if ON_OFF_vector[j] == 1:
            aux=Power[j]
            soma=soma+aux
    sum_power[i]=soma
    soma=0
    
    return(sum_power[i])





In [42]:
##########################################################################################
# Optimização: Maximiza o conforto (pesos).
# Constraint: A soma de potência para cada periodo tem de ser menor do que a Potência max.
##########################################################################################

weight = [10] * len(Index)
ON_OFF = [0] * len(Index)
model = pe.ConcreteModel()

#Declaration
model.loads = pe.Set(initialize=np.arange(0,15))
model.limite = pe.Param(initialize=power_limit)
model.Power = pe.Param(range(len(Index)),initialize=Power)
model.Peso =pe.Param(range(len(Index)),initialize=weight,mutable=True)
model.y =pe.Param(range(len(Index)),initialize=ON_OFF,mutable=True)
model.x = pe.Var(range(len(Index)),domain=pe.Binary, initialize={i: 1 for i in range(len(Index))})

"""
# Prints de verificação
for i in range(0, 15):
    print(f"Index {i+1}: Power {i+1} = {(model.Power[i])}, Peso = {(model.Peso[i])} ")

print({pe.value(model.limite)})

for i in range(len(Index)):
    print(f" x[{i}] = {(model.x[i].value)}")
"""
# create constraint for each x value to be 0 if y is 0
#def _constraint_one(m, j):
#    return m.x[j] <= m.y[j]
#model.constraint_one = pe.Constraint(model.loads, rule=_constraint_one)



#create a constraint model.Power[j]*m.y[j] = 0 if m.x[j] = 0
#def _constraint_two(m, j):
#    return m.Power[j] == 
#model.constraint_two = pe.Constraint(model.loads,rule=_constraint_two)

#Constrait 3
def _constraint_four(m):
   return sum((m.Power[j]*m.x[j]) for j in range(len(Index))) <= m.limite
model.constraint_four = pe.Constraint(rule=_constraint_four)

#Objective function
def _objective_function(m):
    return sum(m.Peso[j]*m.x[j] for j in range(len(Index)))
model.objective = pe.Objective(expr=_objective_function, sense=pe.maximize)

#vector_x = model.x()
#Peso_max = model.objective()
#print(vector_x)
#print(Peso_max)

In [43]:
###########################################################################################################
# Objetivo: Resolve problema em Real-Time
# Funções usadas: 
#                   ->function_weight (dá pesos e faz updates dos mesmos);
#                   ->function_soma (soma as potencias de cada carga para ti);
#                   ->optimização (decide quais cargas estão ON ou OFF, maximizando o conforto);
# Input: Informação de cada carga, dataset com ON/OFF.
# Output: Optimização do dataset.
###########################################################################################################


#Resolução do problema em Real-Time
for i in range (t): 
    update=0
    ON_OFF_vector = function_dataset (i, update, ON_OFF_before)
    ON_OFF_matrix[i] = ON_OFF_vector.copy()
    
    weight = function_weight(i,ON_OFF_vector, update, weight_before, ON_OFF_matrix)             #Vetor com os pesos de cada carga para ti
    weight_matrix [i] = weight.copy()

    for j in range(len(Index)):                                                       #Update model.peso values with new weights
        model.Peso[j] = weight[j]
        model.y[j] = ON_OFF_vector[j]

    def _constraint_three(m, j):
        if weight[j]<0:
            return m.x[j] == m.y[j]
        else:
            return m.x[j] <= m.y[j]
    model.constraint_three = pe.Constraint(model.loads, rule=_constraint_three)

    soma[i] = function_soma(i, ON_OFF_vector)                                             #Soma das potências antes da optimiz para cada ti


    if soma[i] > power_limit:                                                         #Caso a potência seja maior do que o limite:
        results = pe.SolverFactory('scip', executable='C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe').solve(model)
        model.pprint()

        print(f"\nPeriodo {i+1}: Resultados\n")
        for j in range(0, 15):
            print(f"Index {j+1}: x[{j}] = {(model.x[j].value)}")
            print(f"Index {j+1}: y[{j}] = {(model.y[j].value)}")
            print(f"Index {j+1}: ON/OFF = {(ON_OFF_vector[j])}\n")
            #print(f"Peso Max = {model.objective()}") 
            vector_x [j] = np.abs(model.x[j].value)                                    #Output da optimização: Vetor_x 
            #compare[j] = ON_OFF_vector[j] + vector_x[j]                                 #Vetor com informação de alteração do estado de carga
        
        Peso_max = model.objective()                                                   #Output da optimização: Peso_max

        update=1                                                                    #Variavel que entra na função function_weight para atualizar os pesos
        ON_OFF_before[i] = ON_OFF_vector.copy()
        ON_OFF_vector = vector_x.copy()
        ON_OFF_matrix[i] = ON_OFF_vector.copy()
        function_dataset (i, update, ON_OFF_before)                                                 #Atualiza estado da carga com o resultado da optimização
        
                                                                              
        weight_before[i] = weight.copy()                                                      #Guarda os pesos antes da optimização
        weight = function_weight(i,ON_OFF_vector,  update, weight_before, ON_OFF_matrix)
        weight_matrix [i] = weight

        soma_optim[i] = function_soma(i, ON_OFF_vector)                                     #Soma das potências apos a optimiz para cada ti
       

        


    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
    unknown with a new Component (type=<class
    'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block
 

In [44]:
##############################################################################
# Objetivo: ->Acrescentar os minutos e index á matrix ON_OFF e weights;
#           ->Acrescentar a soma das potencia à matriz ON_or_OFF.
##############################################################################


col = []
for i in range(t):
    col.append(i)


for i in range(t):
    ON_OFF_matrix[i].insert(0, col[i])
    ON_OFF_before[i].insert(0, col[i])
    weight_matrix[i].insert(0, col[i])
    weight_before[i].insert(0, col[i])
    

for i in range(t):
    ON_OFF_matrix[i].append(0)
    ON_OFF_matrix[i].append(0)
    #change value in ON_or_OFF list in position 16 for soma[i]
    ON_OFF_matrix[i][16] = soma[i]
    ON_OFF_matrix[i][17] = soma_optim[i]
    

row = []
for i in range(len(Index)+3):
    row.append(i)
row [0] = 'Min/Index'
row [16] ='Sum Power'
row [17] ='Power Optim'
row1 = []
for i in range(len(Index)+1):
    row1.append(i)
row1 [0] = 'Min/Index'


ON_OFF_matrix.insert(0, row)
ON_OFF_before.insert(0, row1)
weight_matrix.insert(0, row1)
weight_before.insert(0, row1)

In [45]:
############################################################################################################
# Objetivo: Imprime as 4 worksheets.
# Função: Escreve no excel: 
#               ->tabela com ON/OFF optimizado para cada load para ti;
#               ->tabela com ON/OFF antes da optimização;
#               ->tabela com pesos optimizados;
#               ->tabela com pesos antes da optimização;
############################################################################################################


# Create a Pandas Excel writer using XlsxWriter
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')

df = pd.DataFrame(ON_OFF_matrix)
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Dataset', index=False, header=False)

df2 = pd.DataFrame(ON_OFF_before)
df2.to_excel(writer, sheet_name='Dataset before', index=False, header=False)

df3 = pd.DataFrame(weight_matrix)
df3.to_excel(writer, sheet_name='Weight', index=False, header=False)

df4 = pd.DataFrame(weight_before)
df4.to_excel(writer, sheet_name='Weight before', index=False, header=False)

writer.save()



C:\Users\anaol\AppData\Local\Temp\ipykernel_4092\3430102654.py:27: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
